In [ ]:
# Do our imports

import nucleardice
import pandas
import plotly.express as px

from typing import Sequence

In [ ]:
def generate_dataframe(max_number_of_dice: int,
                       die_sizes: Sequence[int],
                       trials: int = 1000) -> pandas.DataFrame:
    """Generate a dataframe with data from our generated statistical sample."""
    color: dict[str] = {}
    dsl = list(die_sizes)
    for idx, val in enumerate(dsl):
        pallette = px.colors.sequential.Plasma
        step = int(len(pallette)/len(dsl))
        color[val] = pallette[idx*step]

    shape = {
        "fair_die_expectation": "square",
        "nuclear_median": "circle",
        "nuclear_mean": "diamond",
    }
    outcomes = pandas.DataFrame(
        columns = [
            "n",
            "x",
            "value",
            "measurement",
            "color",
            "shape",
        ]
    )  # Where we will put our results

    for x in die_sizes:
        for n in range(1, max_number_of_dice+1):
            st = nucleardice.NuclearDiceStatistics(n=n, x=x, rolls=trials)
            st.roll()
            item_expected = {
                "n": n,
                "x": x,
                "value": n * ( x + 1 ) / 2,  # Mean and median of fair dice should be the same
                "measurement": "fair_die_expectation",
                "color": color[x],
                "shape": shape["fair_die_expectation"]
            }
            item_mean = {
                "n": n,
                "x": x,
                "value": st._sum_stats.mean,
                "measurement": "nuclear_mean",
                "color": color[x],
                "shape": shape["nuclear_mean"],
            }
            item_median = {
                "n": n,
                "x": x,
                "value": st._sum_stats.median,
                "measurement": "nuclear_median",
                "color": color[x],
                "shape": shape["nuclear_median"]
            }
            outcomes.loc[len(outcomes)] = item_expected
            outcomes.loc[len(outcomes)] = item_mean
            outcomes.loc[len(outcomes)] = item_median
    return outcomes

In [ ]:
outcomes = generate_dataframe(die_sizes = (4, 6, 8, 10, 12, 20), max_number_of_dice=10)

In [ ]:
fig=px.scatter_3d(
    data_frame=outcomes,
    x='x',y='n',z='value',
    color='color', color_discrete_map='identity',
    symbol='shape', symbol_map='identity',
    hover_data=['measurement'],
    width=1200, height=800
)
fig.show()

In [ ]:
# Now let's do the crazy outliers that happen with tiny dice
small_die_outcomes = generate_dataframe(die_sizes = (2, 3), max_number_of_dice=6)
fig=px.scatter_3d(
    data_frame=small_die_outcomes,
    x='x',y='n',z='value',
    color='color', color_discrete_map='identity',
    symbol='shape', symbol_map='identity',
    hover_data=['measurement'],
    width=1200, height=800
)
fig.show()